In [0]:
! pip install kmc2

In [0]:
! yes | pip uninstall opencv-python
! yes | pip uninstall opencv-contrib-python
! yes | pip install opencv-python==3.4.2.16
! yes | pip install opencv-contrib-python==3.4.2.16

In [0]:
! pip install imutils

In [0]:
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import DictionaryLearning
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparseCoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from imutils import paths

import matplotlib.pyplot as plt
import scipy.cluster.vq as vq
import _pickle as cPickle
import pandas as pd
import numpy as np
import imutils
import time
import math
import copy
import kmc2
import cv2 
import os

## 데이터 불러오기

In [0]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle -v

In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! A | unzip 2019-ml-finalproject.zip

In [0]:
caltech_dir = "/content/train"
lists = paths.list_images(caltech_dir)

In [0]:
X = []
y = []

for l in lists:
  path = os.path.dirname(l)
  # if l[15] != 'B':
  X.append(cv2.imread(l))
  y.append(os.path.basename(path))

# categories.remove('BACKGROUND_Google') # it contains all kinds of images. 
print(len(X))

In [0]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

In [0]:
def extract_denseSIFT(img):
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = 4
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img_gray.shape[0], disft_step_size)
                for x in range(0, img_gray.shape[1], disft_step_size)]

    descriptors = sift.compute(img_gray, keypoints)[1]

    return descriptors

In [0]:
import itertools

def getDescriptors(X):
    descriptors=list()
    
    for image in X:
        des = extract_denseSIFT(image)
        descriptors.append(des)
        # print(len(kp))
        
    #flatten list       
    descriptors = list(itertools.chain.from_iterable(descriptors))
    #list to array
    descriptors = np.asarray(descriptors)

    return descriptors

In [0]:
descriptors = getDescriptors(X)

In [0]:
def kMeansDictionary(training, k):
    seeding = kmc2.kmc2(training, k)
    est = MiniBatchKMeans(n_clusters=k,init=seeding).fit(training)

    return est

In [0]:
visualDictionary = kMeansDictionary(descriptors, 200)

In [0]:
def VLAD(X,visualDictionary):

    predictedLabels = visualDictionary.predict(X)
    centers = visualDictionary.cluster_centers_
    labels=visualDictionary.labels_
    k=visualDictionary.n_clusters
   
    m,d = X.shape
    V=np.zeros([k,d])
    #computing the differences

    # for all the clusters (visual words)
    for i in range(k):
        # if there is at least one descriptor in that cluster
        if np.sum(predictedLabels==i)>0:
            # add the diferences
            V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)
    

    V = V.flatten()
    # power normalization, also called square-rooting normalization
    V = np.sign(V)*np.sqrt(np.abs(V))

    # L2 normalization

    V = V/np.sqrt(np.dot(V,V))
    return V

In [0]:
def getVLADDescriptors(X,visualDictionary):
    descriptors=list()
    # idImage =list()
    for img in X:
        kp,des = extract_denseSIFT(img)
        v = VLAD(des,visualDictionary)
        descriptors.append(v)
        # idImage.append(imagePath)
                    
    #list to array    
    descriptors = np.asarray(descriptors)
    return descriptors # , idImage

In [0]:
V = getVLADDescriptors(X, visualDictionary)

In [0]:
all_train_desc = np.vstack((descriptor for descriptor in V))

In [0]:
clf = LinearSVC(random_state=0, C=0.001)
clf.fit(all_train_desc, y)

In [0]:
caltech_dir = "/content/testAll_v2"
lists = paths.list_images(caltech_dir)

X_final = []
img_name = []
for l in lists:
  path = os.path.dirname(l)
  X_final.append(cv2.imread(l))
  img_name.append(l[-14:])

X_final = np.array(X_final)

In [0]:
f_VLAD_DS = getVLADDescriptors(X_final, visualDictionary)

In [0]:
f_all_train_desc = np.vstack((descriptor for descriptor in f_VLAD_DS))

In [0]:
y_predict = clf.predict(f_all_train_desc)

In [0]:
df = pd.read_csv('/content/Label2Names.csv')
df = np.array(df)

result = []

for i in range(len(y_predict)):
  flag = 0
  if y_predict[i] == 'BACKGROUND_Google':
    flag = 1
    result.append(101)
  else:
    for j in range(len(df)):
        if y_predict[i] == df[j][1]:
          result.append(df[j][0])
          flag = 1
  
  if flag != 1:
    result.append(1)

result = np.array(result)
img_name = np.array(img_name)
result = result.reshape(-1, 1)
img_name = img_name.reshape(-1, 1)
total_result = np.hstack([img_name, result])
print(total_result)

In [0]:
df = pd.DataFrame(total_result, columns=['Id', 'Category'])
df.to_csv('results-mjkim-v4.csv', index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-mjkim-VLAD.csv -m "mm-20191123"

In [0]:
with open('/content/VLAD_des.pickle', 'wb') as f:
    cPickle.dump(V, f)

In [0]:
with open('/content/f_VLAD_des.pickle', 'wb') as f:
    cPickle.dump(f_VLAD_DS, f)